In [1]:
import pandas as pd
import altair as alt

In [2]:
def prep_data(df,name):      
    df['name'] = name
    df_te=df[['mol','name']].copy()
    
    df['delta1']  = df['v2']-df['v1']
    df['delta2']  = df['v3']-df['v2']
    df['delta3']  = df['vref']-df['v3']
    
    df_te['total_effect'] = df['vref']-df['v1']
    
    df = df.drop(['v1','v2','v3','vref'], axis=1)
    df = df.melt(id_vars =['mol', 'name'])
    
    df_te = df_te.melt(id_vars =['mol', 'name'])
    
    return df,df_te

In [10]:
df1=pd.read_csv('data1.csv')

In [3]:
df1,df1_te=prep_data(df1,'set1')

In [ ]:
df2=pd.read_csv('data2.csv')

In [4]:
df2, df2_te=prep_data(df2,'set2')

In [5]:
df_plot = pd.concat([df1.set_index('mol'),
                     df2.set_index('mol')]).reset_index()
df_plot_te = pd.concat([df1_te.set_index('mol'),
                     df2_te.set_index('mol')]).reset_index()

In [6]:
df_plot['variable'].replace({'delta1': '\u0394'+'(1)',
                             'delta2': '\u0394'+'(2)',
                             'delta3': '\u0394'+'(3)',
                             'delta4': '\u0394'+'(4)'
                            },inplace=True)

In [7]:
df_plot_all = pd.merge(df_plot, df_plot_te, on=['mol','name'])

In [9]:
order_mol=['Cr', 'Mn', 'Co', 'Zn', 'Mo', 'Tc', 'Ru', 'Pd', 'Ag', 'W', 'Re', 'Pt']
order_where=['set1','set2']


bars=alt.Chart(df_plot_all).mark_bar(size=15).encode(     

    # which field to group columns on
    x=alt.X('name:O',
            axis=alt.Axis(grid=True,labelFontSize=8),
            sort=order_where,
            title=None),

    # which field to use as Y values and how to calculate
    y=alt.Y('value_x:Q',
            axis=alt.Axis(grid=True,title=None)),

    # which field to color by & legend
    color=alt.Color('variable_x',
                    scale=alt.Scale(range=['#4381d1', '#47c488', '#ff6f69']),
                    legend=alt.Legend(title="Contributions",
                                      orient="right",
                                      direction="horizontal",
                                      offset=-200,
                                      titleFontSize=16,
                                      labelFontSize=14)),
                   
    # how to order the data on bars
    order=alt.Order('variable_x:Q', sort='ascending'))


# use separate marks for the 'total effect'
rules = alt.Chart(df_plot_all).mark_tick(color='black', thickness=1.5,size=15,).encode(
        x=alt.X('name:O',axis=alt.Axis(grid=True,title=None)),
        y=alt.Y('value_y:Q',axis=alt.Axis(grid=True,title=None)))


# combine all together
alt.layer(
    bars,
    rules
).properties(height=450,width=50).facet(column=alt.Column('mol',
                      sort=order_mol,
                      header=alt.Header(title='Test plot',
                                        orient='bottom',
                                        titleFontSize=24,
                                        labelFontSize=14,
                                        labelBaseline='line-top',
                                        labelAlign='center',
                                        labelAnchor='middle'))                    
                                       ).resolve_scale(x='independent').configure_view(strokeOpacity=0)

alt.FacetChart(...)